In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

In [52]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import classification_report
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tqdm import tqdm

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [14]:
class_names = ['not_weapon','weapon']
class_names_label = {class_names: i for i, class_names in enumerate(class_names)}

nb_class = len(class_names)

print(class_names_label)

IMAGE_SIZE = (224,224)

{'not_weapon': 0, 'weapon': 1}


In [46]:
def load_data():
    DIRECTORY = r"D:\Github\ReSell\backend\product_classification\data\weapon_image_dataset\weapon_detection"
    CATAGORY = ["test","train"]
    output = []
    for catagory in CATAGORY:
        path = os.path.join(DIRECTORY, catagory)
        print(path)
        images = []
        labels = []
        
        print("Loading {}".format(catagory))
        for folder in os.listdir(path):
            label = class_names_label[folder]
            
            for file in os.listdir(os.path.join(path, folder)):
                img_path = os.path.join(os.path.join(path, folder), file)
                #print(img_path)
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, IMAGE_SIZE)
                
                images.append(image)
                labels.append(label)
        images = np.array(images, dtype='float32')
        labels = np.array(labels, dtype='int32')
        output.append((images,labels))
    return output

In [47]:
(test_images, test_labels), (train_images,train_labels) = load_data()

D:\Github\ReSell\backend\product_classification\data\weapon_image_dataset\weapon_detection\test
Loading test
D:\Github\ReSell\backend\product_classification\data\weapon_image_dataset\weapon_detection\train
Loading train


In [50]:
train_images, train_labels = shuffle(train_images, train_labels, random_state=25)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet50(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Replace final layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

model = model.to(device)

c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\ADMIN/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:17<00:00, 5.70MB/s]


In [53]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

In [56]:
from tensorflow.keras.applications import ResNet50
base_model = ResNet50(weights='imagenet',include_top=False, input_shape=(224, 224, 3), classes=2)
for layer in base_model.layers:
    layer.trainable = False
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)  
ResNet50_model = Model(inputs=base_model.input, outputs=output)

In [58]:
ResNet50_model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = ResNet50_model.fit(train_images, train_labels, batch_size=64 , epochs=20, validation_split=0.2)

Epoch 1/20

10/17 [================>.............] - ETA: 11s - loss: 25.8714 - accuracy: 0.7078

In [ ]:
plot_accuracy_loss(history)